- BÀI TẬP SQL CHƯƠNG 3
- Họ và Tên: Đặng Hữu Tâm
- stt: 35
- MSV: 22174600022
- Lớp: DHKL16A2HN

Câu 1: Viết câu lệnh sql để tính sự tương quan giữa A và B theo công thứcthức

In [17]:
import pandas as pd
import scipy.stats as stats
import sqlite3
import math

In [18]:
# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Đăng ký hàm SQRT cho SQLite
conn.create_function("SQRT", 1, math.sqrt)

# Tạo bảng và chèn dữ liệu
cursor.execute('''
    CREATE TABLE data_table (
        A REAL,
        B REAL
    )
''')

data = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]
cursor.executemany("INSERT INTO data_table (A, B) VALUES (?, ?)", data)
conn.commit()

# Truy vấn Pearson đơn giản
query = '''
SELECT
    (COUNT(*) * SUM(A*B) - SUM(A)*SUM(B)) * 1.0 /
    (SQRT(COUNT(*) * SUM(A*A) - SUM(A)*SUM(A)) *
     SQRT(COUNT(*) * SUM(B*B) - SUM(B)*SUM(B))) AS pearson_correlation
FROM data_table;
'''

cursor.execute(query)
correlation = cursor.fetchone()[0]

print("Hệ số tương quan Pearson (SQL rút gọn):", correlation)

# Đóng kết nối
cursor.close()
conn.close()


Hệ số tương quan Pearson (SQL rút gọn): 0.9999999999999999


Đây là một giá trị gần với 1, cho thấy mối tương quan tuyến tính dương gần như hoàn hảo giữa hai biến A và B.

Câu 2: Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày, và chấm điểm theo thang từ 1 đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không? Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2

In [ ]:
# Kết nối SQLite 
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute("""
    CREATE TABLE car_tests (
        Day TEXT,
        Model TEXT,
        Score REAL
    )
""")

# Dữ liệu thử nghiệm
data = [
    ("Day 1", "A", 8), ("Day 1", "B", 9), ("Day 1", "C", 7),
    ("Day 2", "A", 7.5), ("Day 2", "B", 8.5), ("Day 2", "C", 7),
    ("Day 3", "A", 6), ("Day 3", "B", 7), ("Day 3", "C", 8),
    ("Day 4", "A", 7), ("Day 4", "B", 6), ("Day 4", "C", 5)
]

# Chèn dữ liệu vào bảng
cursor.executemany("INSERT INTO car_tests VALUES (?, ?, ?)", data)
conn.commit()
# Truy vấn dữ liệu từ SQLite
query = "SELECT Day, Model, Score FROM car_tests"
df = pd.read_sql_query(query, conn)

# Chuyển dữ liệu thành bảng tần suất 
table = pd.pivot_table(df, values="Score", index="Day", columns="Model", aggfunc="sum")
# Kiểm định Chi-square
chi2_stat, p_value, dof, expected = stats.chi2_contingency(table)

# In kết quả
print(f"Chi-square Statistic: {chi2_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(pd.DataFrame(expected, index=table.index, columns=table.columns))
# Kiểm tra kết quả
alpha = 0.05
if p_value < alpha:
    print("Có sự khác biệt đáng kể giữa các mẫu xe (bác bỏ H0).")
else:
    print("Không có đủ bằng chứng để khẳng định sự khác biệt giữa các mẫu xe (chấp nhận H0).")

# Đóng kết nối SQLite
conn.close()

Chi-square Statistic: 0.826439691310499
P-value: 0.9913459315666546
Degrees of Freedom: 6
Expected Frequencies:
Model         A         B         C
Day                                
Day 1  7.953488  8.511628  7.534884
Day 2  7.622093  8.156977  7.220930
Day 3  6.959302  7.447674  6.593023
Day 4  5.965116  6.383721  5.651163
Không có đủ bằng chứng để khẳng định sự khác biệt giữa các mẫu xe.


Câu 3: Bảng flights (departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian

In [ ]:
# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng flights
cursor.execute("CREATE TABLE flights (id INTEGER PRIMARY KEY, departure_time INTEGER)")

# Chèn dữ liệu mẫu
data = [(1, 830), (2, 1445), (3, 1230), (4, 700), (5, 2200)]
cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", data)
conn.commit()

# Truy vấn và chuyển đổi thời gian
query = """
SELECT id, departure_time,
       printf('%02d:%02d %s', 
              departure_time / 100 % 12, 
              departure_time % 100, 
              CASE WHEN departure_time / 100 >= 12 THEN 'PM' ELSE 'AM' END
       ) AS formatted_time
FROM flights
"""
cursor.execute(query)

# Hiển thị kết quả
for row in cursor.fetchall():
    print(row)

# Đóng kết nối
conn.close()

(1, 830, '08:30 AM')
(2, 1445, '02:45 PM')
(3, 1230, '00:30 PM')
(4, 700, '07:00 AM')
(5, 2200, '10:00 PM')


Câu 4: Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.

In [ ]:
# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute("CREATE TABLE data (id INTEGER PRIMARY KEY, value REAL)")

# Thêm dữ liệu mẫu
data_values = [(1, 10), (2, 12), (3, 15), (4, 14), (5, 11), 
               (6, 13), (7, 50), (8, 14), (9, 12), (10, 13)]
cursor.executemany("INSERT INTO data (id, value) VALUES (?, ?)", data_values)
conn.commit()

# Truy vấn tính toán MAD và phát hiện ngoại lệ
query = """
WITH median_calc AS (
    SELECT value 
    FROM data
    ORDER BY value
    LIMIT 1 OFFSET (SELECT COUNT(*) / 2 FROM data)
),
stats AS (
    SELECT 
        value,
        ABS(value - (SELECT value FROM median_calc)) AS deviation
    FROM data
),
mad_table AS (
    SELECT AVG(deviation) AS mad FROM stats
)
SELECT d.id, d.value 
FROM data d, mad_table, median_calc
WHERE ABS(d.value - median_calc.value) > 1.5 * mad_table.mad;
"""
cursor.execute(query)

# Hiển thị ngoại lệ
outliers = cursor.fetchall()
print("Ngoại lệ:", outliers)

# Đóng kết nối
conn.close()

Ngoại lệ: [(7, 50.0)]


Câu 5: Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”.

In [ ]:
# Kết nối đến cơ sở dữ liệu SQLite (thay 'patients.db' bằng tên file cơ sở dữ liệu của bạn)
conn = sqlite3.connect('patients.db')
cursor = conn.cursor()

# Tạo bảng Patient
cursor.execute('''
CREATE TABLE IF NOT EXISTS Patient (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    last_name TEXT,
    weight REAL,
    height REAL
)
''')

# Chèn một số dữ liệu mẫu vào bảng Patient
cursor.execute('''
INSERT OR IGNORE INTO Patient (last_name, weight, height)
VALUES ('Nguyen', 70, 175),
       ('Tran', 65, 160),
       ('Nguyen', 70, 175),
       ('Le', 80, 180)
''')

# Thực hiện truy vấn SQL để so sánh các bệnh nhân
cursor.execute('''
SELECT p1.id AS patient_id1, p2.id AS patient_id2,
       CASE 
           WHEN p1.last_name = p2.last_name AND p1.weight = p2.weight THEN 'Cả hai là một người'
           ELSE 'Đây là hai người khác nhau'
       END AS comparison_result
FROM Patient p1, Patient p2
WHERE p1.id < p2.id
''')

results = cursor.fetchall()
for row in results:
    print(f"Patient {row[0]} vs Patient {row[1]}: {row[2]}")

# Đóng kết nối
conn.commit() 
conn.close()

Patient 1 vs Patient 2: Đây là hai người khác nhau
Patient 1 vs Patient 3: Cả hai là một người
Patient 1 vs Patient 4: Đây là hai người khác nhau
Patient 2 vs Patient 3: Đây là hai người khác nhau
Patient 2 vs Patient 4: Đây là hai người khác nhau
Patient 3 vs Patient 4: Đây là hai người khác nhau
